In [8]:
RKP = "DL031"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set_theme(style="darkgrid")

In [17]:
def loadcsv():
    met = pd.read_csv('./data/rkpuram.csv',delimiter=';',skiprows=24)
    aqi = pd.read_csv('./data/station_hour.csv')
    return met,aqi
met,aqi = loadcsv()

C:\Users\confusement\miniconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
class dataset:
    def __init__(self,load=True,city="RKP"):
        if(load,met,aqi):
            self.metro_data = met
            self.aqi_data = aqi
            self.metro_data.rename(columns={'# Date': 'Date',}, inplace=True)
            self.metro_data.rename(columns={'UT time': 'Time',}, inplace=True)
            self.aqi_data['Time'] = self.aqi_data['Datetime'].str[-8:-3]
            self.aqi_data['Date'] = self.aqi_data['Datetime'].str[0:10]
            stations = ["DL"+str(x).zfill(3) for x in range(1,39)]
            self.split_aqi = {}
            for i in range(len(stations)):
                self.split_aqi[stations[i]] = (self.aqi_data[self.aqi_data['StationId'] == stations[i]])
        else:
            self.metro_data = None
            self.aqi_data = None
    def getdf(self,station):
        return self.split_aqi[station]
    def plot(self,station):
        df = self.getdf(station)
        
    def stats(self):
        pass
dat = dataset()

In [19]:
dat.getdf('DL031')

,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Time,Date
1000314,DL031,2015-04-04 11:00:00,59.67,39.17,5.83,41.58,46.47,41.97,0.33,13.53,79.50,1.61,7.09,NaN,NaN,NaN,11:00,2015-04-04
1000315,DL031,2015-04-04 12:00:00,37.67,43.00,7.24,36.19,43.51,38.40,0.32,17.91,65.00,1.87,6.91,NaN,NaN,NaN,12:00,2015-04-04
1000316,DL031,2015-04-04 13:00:00,36.00,58.00,5.54,26.33,32.07,26.73,0.42,6.30,58.17,1.36,6.26,NaN,NaN,NaN,13:00,2015-04-04
1000317,DL031,2015-04-04 14:00:00,NaN,48.00,7.12,22.67,30.88,20.23,0.40,NaN,54.42,0.97,4.21,NaN,NaN,NaN,14:00,2015-04-04
1000318,DL031,2015-04-04 15:00:00,32.00,50.33,7.02,19.72,28.03,18.80,0.45,NaN,49.92,1.30,4.83,NaN,NaN,NaN,15:00,2015-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046259,DL031,2020-06-30 20:00:00,28.00,95.00,1.57,22.17,13.07,14.93,0.97,18.45,11.53,NaN,NaN,NaN,64.0,Satisfactory,20:00,2020-06-30
1046260,DL031,2020-06-30 21:00:00,28.00,90.00,2.00,24.45,14.62,15.15,1.02,11.27,10.25,NaN,NaN,NaN,67.0,Satisfactory,21:00,2020-06-30
1046261,DL031,2020-06-30 22:00:00,28.00,96.00,2.10,19.92,12.33,15.32,1.00,8.78,10.05,NaN,NaN,NaN,68.0,Satisfactory,22:00,2020-06-30
1046262,DL031,2020-06-30 23:00:00,27.00,90.00,1.60,21.98,13.03,13.32,0.93,7.25,10.57,NaN,NaN,NaN,70.0,Satisfactory,23:00,2020-06-30
